In [47]:
import os

In [48]:
%pwd


'C:\\DL\\Project_Resume\\Chicken_Disease_Classification'

In [49]:
os.chdir("../")

In [50]:
%pwd

'C:\\DL\\Project_Resume'

In [51]:
#data config entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [52]:
from Chicken_Disease_Classification.constants import *
from Chicken_Disease_Classification.utils.common import read_yaml, create_directories

In [53]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [54]:

import os
import urllib.request as request
import zipfile
from Chicken_Disease_Classification import logger
from Chicken_Disease_Classification.utils.common import get_size

In [55]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [56]:
os.chdir("C:\DL\Project_Resume\Chicken_Disease_Classification")

In [57]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-03 05:03:45,175] : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-09-03 05:03:45,177] : INFO : common : yaml file: params.yaml loaded successfully]
[2024-09-03 05:03:45,179] : INFO : common : created directory at: artifacts]
[2024-09-03 05:03:45,180] : INFO : common : created directory at: artifacts/data_ingestion]
[2024-09-03 05:03:48,108] : INFO : 738703821 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6557:283FA9:3478EB:4429C5:66D64B58
Accept-Ranges: bytes
Date: Mon, 02 Sep 2024 23:33:47 GMT
Via: 1.1 varnish
X-Served-By: cache-bom